In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
model_id = "microsoft/DialoGPT-medium"
peft_model_id = "hibana2077/DialoGPT-medium-PTT-CUSTOM"

In [1]:


model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

c:\Program Files\Python310\lib\site-packages\peft\tuners\lora\model.py:347: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

In [2]:
#save model as pytorch_model.bin
import torch

torch.save(model.state_dict(), "pytorch_model.bin")

In [8]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D()
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1024, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=3072, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
 

In [10]:
# TEST MODEL

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
tokenizer.padding_side = "left"
model = AutoModelForCausalLM.from_pretrained(model_id)
model.load_adapter(peft_model_id)


# Let's chat for 5 lines

for step in range(1):

    input_text = input("User: ")
    print("User: {}".format(input_text))
    print(tokenizer.encode( input_text + tokenizer.eos_token))

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode( input_text + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


User: What is apple?
[2061, 318, 17180, 30, 50256]
DialoGPT: A fruit.
